<a href="https://colab.research.google.com/github/bokovaj/MMVSULAB1/blob/main/%D0%9C%D0%B0%D1%82%20%D1%82%D0%B5%D0%BE%D1%80%D0%B8%D1%8F%20%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Домашнее задание 1


In [1]:
import numpy as np
import plotly.graph_objs as go
X = np.array([
    [1, -1, -1],
    [1, -1, 1],
    [1, 1, -1],
    [1, 1, 1]
])
X_xor = np.array([
    [1, -1, -1, -1],
    [1, -1, 1, -1],
    [1, 1, -1, -1],
    [1, 1, 1, 1]
])
Y_and = np.array([
    [-1],
    [-1],
    [-1],
    [1]
])
Y_or = np.array([
    [-1],
    [1],
    [1],
    [1]
])
Y_xor = np.array([
    [-1],
    [1],
    [1],
    [-1]
])
# Область определения (0, 1)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_back(x):
    return x * (1 - x)
# Область определения (-1, 1)
def tanh(x):
    return np.tanh(x)
def tanh_back(x):
    return 1 - x**2
# функция разделяющей линии
def line_(x1, W):
    return (-x1 * W[1] - W[0]) / W[2]
# функция для создания анимации
def settings_animation(W, function_learning: str = 'xor'):
  points_x1 = [-1, -1, 1, 1]
  points_x2 = [-1, 1, -1, 1]
  x1 = np.arange(-1.25, 1.25, 0.1)
  fig_dict = {
      "data": [],
      "layout": {},
      "frames": []
  }
  fig_dict["layout"]["hovermode"] = "closest"
  fig_dict["layout"]["updatemenus"] = [
      {
          "buttons": [
              {
                  "args": [None, {"frame": {"duration": 1000, "redraw": False},
                                  "fromcurrent": True, "transition": {"duration": 300,
                                                                      "easing": "quadratic-in-out"}}],
                  "label": "Play",
                  "method": "animate"
              },
              {
                  "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}],
                  "label": "Pause",
                  "method": "animate"
              }
          ],
          "direction": "left",
          "pad": {"r": 10, "t": 87},
          "showactive": False,
          "type": "buttons",
          "x": 0.1,
          "xanchor": "right",
          "y": 0,
          "yanchor": "top"
      }
  ]
  sliders_dict = {
      "active": 0,
      "yanchor": "top",
      "xanchor": "left",
      "currentvalue": {
          "font": {"size": 20},
  #         "prefix": "Epoch:",
          "visible": True,
          "xanchor": "right"
      },
      "transition": {"duration": 300, "easing": "cubic-in-out"},
      "pad": {"b": 10, "t": 50},
      "len": 0.9,
      "x": 0.1,
      "y": 0,
      "steps": []
  }
  data_dict_points = {
      "x": points_x1,
      "y": points_x2,
      "mode": "markers",
      "marker": {
          'color': dict_colors[function_learning][0]
      }
  }
  data_dict = {
      "x": x1,
      "y": line_(x1, W),
      "mode": "lines"
  }
  fig_dict["data"].append(data_dict)
  fig_dict["data"].append(data_dict_points)
  return fig_dict, sliders_dict
# градиентный спуск с выбором аппроксимируемой функции (and, or, xor)
def gradient_descent(W, X, E, batch, fig_dict, sliders_dict, function_learning = 'xor'):
    x1 = np.arange(-1.25, 1.25, 0.1)
    error = np.array([[.5], [.5], [.5], [.5]])
    iterations = int(np.ceil(X.shape[0] / batch))
    for epoch in range(epochs + 1):
        for iteration in range(iterations):

            index_start = iteration * batch
            index_stop = iteration * batch + batch

            frame = {"data": [], "name": str(epoch)+str(iteration)}
            data_dict = {
              "x": x1,
              "y": line_(x1, W),
              "mode": "lines"
            }
            frame["data"].append(data_dict)
            fig_dict["frames"].append(frame)
            slider_step = {"args": [
              [str(epoch)+str(iteration)],
              {"frame": {"duration": 300, "redraw": False},
              "mode": "immediate",
              "transition": {"duration": 100}}
            ],
              "label": f'Epoch: {epoch} Batch: {iteration}',
              "method": "animate"}
            sliders_dict["steps"].append(slider_step)

            #  forward
            Y = tanh(np.dot(X[index_start : index_stop], W))
            Y_round = np.array([np.sign(i) for i in Y]).reshape(-1, 1)
            error_round = dict_colors[function_learning][1][index_start : index_stop] - Y_round
            error[index_start : index_stop] = np.array([dict_colors[function_learning][1][index_start : index_stop][i][0] - Y[i][0] if error_round[i] != 0 else 0 for i in range(batch)]).reshape(-1, 1)

            #  back
            w_delta = error[index_start : index_stop] * tanh_back(Y)
            W += np.dot(X[index_start : index_stop].T, w_delta) * E

            if all(error == 0.):
                fig_dict["layout"]["sliders"] = [sliders_dict]
                return fig_dict
    fig_dict["layout"]["sliders"] = [sliders_dict]
    return fig_dict
list_colors_and = ['red', 'red', 'red', 'green']
list_colors_or = ['red', 'green', 'green', 'green']
list_colors_xor = ['red', 'green', 'green', 'red']
dict_colors = {
    'and': (list_colors_and, Y_and),
    'or': (list_colors_or, Y_or),
    'xor': (list_colors_xor, Y_xor)
}
E = 0.01
epochs = 100
# W = np.array([-0.5, 0.5, 0.5]).reshape(-1,1)
W = np.random.random((3, 1))
batch = 1
function_learning = 'xor'
animation = gradient_descent(W, X, E, batch, *settings_animation(W, function_learning), function_learning)
fig = go.Figure(animation)
fig.show()
